In [1]:
from wandb.keras import WandbCallback
import sys
import time
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from utils.utils_mine import *
import tensorflow as tf
import tensorflow.keras as keras
from datetime import date
import numpy as np
import random
import tensorflow_addons as tfa
import wandb
import gc
from classifiers.classifier_factory import create_classifier
from scripts.plot.DL.read_LOO_nestedCV_gnntr import get_sorted_loo_array
# import pingouin as pg 


/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.14.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issu

In [3]:
data = np.load('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/results/his_metrics.npy', allow_pickle=True)
print(data)

{'fprs': [array([0.  , 0.02, 0.04, 0.08, 0.14, 0.18, 0.18, 0.18, 0.22, 0.28, 0.36,
       0.4 , 0.4 , 0.44, 0.44, 0.5 , 0.5 , 0.52, 0.6 , 0.62, 0.7 , 0.7 ,
       0.72, 0.72, 0.74, 0.74, 0.84, 0.92, 0.96, 0.96, 1.  ]), array([0.  , 0.02, 0.08, 0.08, 0.14, 0.14, 0.34, 0.34, 0.36, 0.36, 0.38,
       0.38, 0.4 , 0.4 , 0.48, 0.48, 0.54, 0.54, 0.64, 0.64, 0.7 , 0.74,
       0.82, 0.82, 0.84, 0.84, 1.  ]), array([0.  , 0.04, 0.68, 0.68, 0.68, 0.68, 0.68, 0.76, 0.82, 0.82, 1.  ]), array([0.  , 0.02, 0.06, 0.1 , 0.14, 0.28, 0.32, 0.36, 0.44, 0.48, 0.48,
       0.5 , 0.54, 0.62, 0.68, 0.68, 0.7 , 0.74, 0.78, 0.9 , 0.9 , 0.96,
       1.  ]), array([0.  , 0.02, 0.12, 0.16, 0.24, 0.24, 0.26, 0.3 , 0.36, 0.36, 0.4 ,
       0.4 , 0.46, 0.46, 0.48, 0.48, 0.52, 0.52, 0.56, 0.58, 0.58, 0.62,
       0.64, 0.68, 0.68, 0.78, 0.82, 0.88, 0.88, 0.92, 0.96, 1.  ]), array([0.  , 0.02, 0.14, 0.14, 0.28, 0.28, 0.36, 0.38, 0.38, 0.42, 0.42,
       0.48, 0.48, 0.5 , 0.5 , 0.52, 0.52, 0.54, 0.54, 0.58, 0.58, 0.6 ,

In [4]:
a = np.ones((4,4))
for i in range(4):
    a[:,i] *= i
    
a[2, :] *= 5
print(a)
a[:,1] = (a[:, 2] + a[:, 3]) / 2
print(a)

[[ 0.  1.  2.  3.]
 [ 0.  1.  2.  3.]
 [ 0.  5. 10. 15.]
 [ 0.  1.  2.  3.]]
[[ 0.   2.5  2.   3. ]
 [ 0.   2.5  2.   3. ]
 [ 0.  12.5 10.  15. ]
 [ 0.   2.5  2.   3. ]]


In [2]:
result = np.load('/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/results/gnn_transformer/prognosis_mix_hb/pretreatment_response/loocv_v4l1_rate_0.01_l2_rate_0.001_d_model_16_batch_size_64_n_layers_6/LOO_nested_CV/y_pred.npy')
print('loocv4 - ', result.shape)

loocv4 -  (320, 2)


In [3]:
allData_loocv4 = np.load('/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/prognosis_mix_hb/pretreatment_response/fnirs_pred_loocv4.npy')
print('allData_loocv4 - ', allData_loocv4.shape)

allData_loocv4 -  (320, 2)
